In [1]:
import copy
import os.path
import sys

pwd = os.getcwd()
sys.path.append(os.path.dirname(pwd))

from utils.model_utils.evaluate import evaluate_model
from utils.model_utils.load_model import *
from utils.model_utils.model_config import ModelConfig
from utils.dataset_utils.load_dataset import load_data
from utils.decompose_utils.weight_remover import WeightRemoverBert
from utils.decompose_utils.concern_identification import ConcernIdentificationBert
from utils.decompose_utils.tangling_identification import TanglingIdentification
from transformers import AutoConfig
from utils.model_utils.save_module import save_module
from datetime import datetime
from utils.decompose_utils.concern_modularization import ConcernModularizationBert
from utils.decompose_utils.sampling import sampling_class
from utils.dataset_utils.load_dataset import convert_dataset_labels_to_binary, extract_and_convert_dataloader
import torch


model_name = "sadickam/sdg-classification-bert"
model_type = "pretrained"
data = "OSDG"
num_labels = 16


# model_name = "textattack/bert-base-uncased-imdb"
# model_type = "pretrained"
# data = "IMDb"
# num_labels = 2

# model_name = "fabriceyhc/bert-base-uncased-yahoo_answers_topics"
# model_type = "pretrained"
# data = "Yahoo"
# num_labels = 10

for i in range(0, 4):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
    
    checkpoint_name = None
    config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
    model_config = ModelConfig(
        _model_name=model_name,
        _model_type=model_type,
        _data=data,
        _transformer_config=config,
        _checkpoint_name=checkpoint_name,
        _device=device,
    )
    
    
    model, tokenizer, checkpoint = load_classification_model(model_config, train_mode=False)
    
    train_dataloader, valid_dataloader, test_dataloader = load_data(
        model_config, batch_size=32
    )
    
    print("Start Time:" + datetime.now().strftime("%H:%M:%S"))
    print("#Module " + str(i) + " in progress....")
    num_samples = 64
    
    positive_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, True, 4, device=device
    )
    negative_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, False, 4, device=device
    )
    
    all_samples = sampling_class(
        train_dataloader, 200, 20, num_labels, False, 4, device=device
    )
    
    print("origin")
    # evaluate_model(model, model_config, test_dataloader)
    
    module = copy.deepcopy(model)
    wr = WeightRemoverBert(model, p=0.9)
    ci = ConcernIdentificationBert(model, p=0.4)
    ti = TanglingIdentification(model, p=0.5)
    
    print("Start Positive CI sparse")
    
    import matplotlib.pyplot as plt
    import numpy as np
    
    def double_plot(tensor1, tensor2):
        tensor_data1 = tensor1.cpu().detach().numpy()
        flatten_data1 = tensor_data1.flatten()
    
        tensor_data2 = tensor2.cpu().detach().numpy()
        flatten_data2 = tensor_data2.flatten()
        
        fig, ax1 = plt.subplots(figsize=(6, 5))
    
        # Plotting the first histogram
        ax1.hist(flatten_data1, bins=40, edgecolor='black', alpha=0.5, label='concern identification', color='b')
        ax1.set_xlabel('Value')
        ax1.set_ylabel('Frequency', color='b')
        ax1.tick_params(axis='y', labelcolor='b')
        
        # Plotting the second histogram
        ax1.hist(flatten_data2, bins=40, edgecolor='black', alpha=0.5, label='tangling identification', color='y')
        ax1.set_ylabel('Frequency', color='g')
        ax1.tick_params(axis='y', labelcolor='g')
    
        plt.title('Distribution of the weight')
        plt.legend()
        plt.grid(True)
        plt.show()
    
    sparsity_list = []
    prec_list = []
    rec_list = []
    f1_list = []
    
    def print_overlapped_weight(model, module):
        top_ff1 = model.bert.encoder.layer[0].intermediate.dense.weight
        bottom_ff1 = model.bert.encoder.layer[-1].intermediate.dense.weight
        top_ff2 = model.bert.encoder.layer[0].output.dense.weight
        bottom_ff2 = model.bert.encoder.layer[-1].output.dense.weight
        pooler = model.bert.pooler.dense.weight
        classifier = model.bert.pooler.dense.weight
    
        top_ff1_m = module.bert.encoder.layer[0].intermediate.dense.weight
        bottom_ff1_m = module.bert.encoder.layer[-1].intermediate.dense.weight
        top_ff2_m = module.bert.encoder.layer[0].output.dense.weight
        bottom_ff2_m = module.bert.encoder.layer[-1].output.dense.weight
        pooler_m = module.bert.pooler.dense.weight
        classifier_m = module.bert.pooler.dense.weight
    
        double_plot(top_ff1, top_ff1_m)
        double_plot(bottom_ff1, bottom_ff1_m)
        double_plot(top_ff2, top_ff2_m)
        double_plot(bottom_ff2, bottom_ff2_m)
        double_plot(pooler, pooler_m)
        double_plot(classifier, classifier_m)
    
    def get_sparsity(module):
        cnt = 0
        for i, layer in enumerate(module.bert.encoder.layer):
            cnt += torch.sum(layer.intermediate.dense.weight == 0).item()
            cnt += torch.sum(layer.output.dense.weight == 0).item()
        cnt += torch.sum(module.bert.pooler.dense.weight == 0).item()
        cnt += torch.sum(module.classifier.weight == 0).item()
        return cnt
    
    def get_report(result, i):
        import re
        if not isinstance(result, str):
            result = str(result)
        pattern = rf'\s+{i}\s+(\d\.\d+)\s+(\d\.\d+)\s+(\d\.\d+)\s+(\d+)'
    
        match = re.search(pattern, result)
        precision = float(match.group(1))
        recall = float(match.group(2))
        f1_score = float(match.group(3))
        return precision, recall, f1_score
    
    eval_step = 1
    for idx, batch in enumerate(all_samples):
        input_ids, attn_mask, _, total_sampled = batch
        ref_module = copy.deepcopy(module)
        with torch.no_grad():
            wr.propagate(module, input_ids)
        if idx % eval_step == 0:
            result = evaluate_model(module, model_config, test_dataloader)
            sparsity_list.append(get_sparsity(module))
            prec, rec, f1 = get_report(result, i)
            prec_list.append(prec)
            rec_list.append(prec)
            f1_list.append(prec)
    print("Start Positive CI after sparse")
    
    for idx, batch in enumerate(positive_samples):
        ref_module = copy.deepcopy(module)
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            ci.propagate(module, input_ids)
        if idx % eval_step == 0:
            result = evaluate_model(module, model_config, test_dataloader)
            sparsity_list.append(get_sparsity(module))
            prec, rec, f1 = get_report(result, i)
            prec_list.append(prec)
            rec_list.append(prec)
            f1_list.append(prec)
    
    print("Start Negative TI")
    
    for idx, batch in enumerate(negative_samples):
        ref_module = copy.deepcopy(module)
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            ti.propagate(module, input_ids)
        if idx % eval_step == 0:
            result = evaluate_model(module, model_config, test_dataloader)
            sparsity_list.append(get_sparsity(module))
            prec, rec, f1 = get_report(result, i)
            prec_list.append(prec)
            rec_list.append(prec)
            f1_list.append(prec)
    
    print(sparsity_list)
    print(prec_list)
    print(rec_list)
    print(f1_list)
    
    # import matplotlib.pyplot as plt
    
    # def plot_sparsity(sparsity_list):
    #     """
    #     모델의 sparsity 그래프를 그리는 함수.
        
    #     Args:
    #     sparsity_list (list of float): sparsity 값들의 리스트.
    #     """
    #     plt.figure(figsize=(10, 6))
    #     plt.plot(sparsity_list, marker='o', linestyle='-', color='b')
    #     plt.title('Model Sparsity Over Time')
    #     plt.xlabel('Iteration')
    #     plt.ylabel('Sparsity')
    #     plt.grid(True)
    #     plt.show()
    
    # plot_sparsity(sparsity_list)





Directory /home/Minwoo/LESN/Decompose/DecomposeBERT/Models/Configs/pretrained/sadickam/sdg-classification-bert exists.
Loading the model.
Start Time:01:26:41
#Module 0 in progress....
origin
Start Positive CI sparse


Evaluating: 100%|█████████████████████████████████████████████████████████████████████| 400/400 [02:40<00:00,  2.49it/s]


Loss: 0.9465
Precision: 0.7790, Recall: 0.7857, F1-Score: 0.7781
              precision    recall  f1-score   support

           0       0.76      0.67      0.71       797
           1       0.84      0.71      0.77       775
           2       0.87      0.88      0.88       795
           3       0.87      0.82      0.85      1110
           4       0.86      0.80      0.83      1260
           5       0.88      0.69      0.77       882
           6       0.85      0.80      0.82       940
           7       0.48      0.61      0.54       473
           8       0.65      0.86      0.74       746
           9       0.61      0.72      0.66       689
          10       0.76      0.78      0.77       670
          11       0.62      0.81      0.71       312
          12       0.72      0.81      0.76       665
          13       0.83      0.86      0.84       314
          14       0.86      0.78      0.82       756
          15       0.97      0.98      0.97      1607

    accuracy   

Evaluating: 100%|█████████████████████████████████████████████████████████████████████| 400/400 [03:05<00:00,  2.16it/s]


Loss: 0.9466
Precision: 0.7789, Recall: 0.7857, F1-Score: 0.7781
              precision    recall  f1-score   support

           0       0.76      0.67      0.71       797
           1       0.84      0.71      0.77       775
           2       0.87      0.88      0.88       795
           3       0.87      0.82      0.85      1110
           4       0.86      0.80      0.83      1260
           5       0.88      0.69      0.77       882
           6       0.86      0.80      0.82       940
           7       0.48      0.61      0.54       473
           8       0.65      0.86      0.74       746
           9       0.61      0.72      0.66       689
          10       0.76      0.78      0.77       670
          11       0.62      0.81      0.71       312
          12       0.72      0.81      0.76       665
          13       0.83      0.86      0.84       314
          14       0.86      0.78      0.82       756
          15       0.97      0.98      0.97      1607

    accuracy   

...

Loss: 0.8971
Precision: 0.7731, Recall: 0.7724, F1-Score: 0.7686
              precision    recall  f1-score   support

           0       0.71      0.67      0.69       797
           1       0.84      0.69      0.76       775
           2       0.85      0.88      0.86       795
           3       0.87      0.81      0.84      1110
           4       0.86      0.80      0.83      1260
           5       0.90      0.69      0.78       882
           6       0.85      0.78      0.81       940
           7       0.50      0.53      0.51       473
           8       0.65      0.85      0.73       746
           9       0.53      0.73      0.62       689
          10       0.78      0.76      0.77       670
          11       0.71      0.76      0.73       312
          12       0.67      0.82      0.74       665
          13       0.83      0.85      0.84       314
          14       0.86      0.77      0.81       756
          15       0.97      0.97      0.97      1607

    accuracy   

Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [04:20<00:00,  1.54it/s]


Loss: 0.8971
Precision: 0.7731, Recall: 0.7724, F1-Score: 0.7686
              precision    recall  f1-score   support

           0       0.71      0.67      0.69       797
           1       0.84      0.69      0.76       775
           2       0.85      0.88      0.86       795
           3       0.87      0.81      0.84      1110
           4       0.86      0.80      0.83      1260
           5       0.90      0.69      0.78       882
           6       0.85      0.78      0.81       940
           7       0.50      0.53      0.51       473
           8       0.65      0.85      0.73       746
           9       0.53      0.73      0.62       689
          10       0.78      0.76      0.77       670
          11       0.71      0.76      0.73       312
          12       0.67      0.82      0.74       665
          13       0.83      0.85      0.84       314
          14       0.86      0.77      0.81       756
          15       0.97      0.97      0.97      1607

    accuracy   

Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [04:22<00:00,  1.53it/s]

Loss: 0.8971
Precision: 0.7731, Recall: 0.7724, F1-Score: 0.7686
              precision    recall  f1-score   support

           0       0.71      0.67      0.69       797
           1       0.84      0.69      0.76       775
           2       0.85      0.88      0.86       795
           3       0.87      0.81      0.84      1110
           4       0.86      0.80      0.83      1260
           5       0.90      0.69      0.78       882
           6       0.85      0.78      0.81       940
           7       0.50      0.53      0.51       473
           8       0.65      0.85      0.73       746
           9       0.53      0.73      0.62       689
          10       0.78      0.76      0.77       670
          11       0.71      0.76      0.73       312
          12       0.67      0.82      0.74       665
          13       0.83      0.85      0.84       314
          14       0.86      0.77      0.81       756
          15       0.97      0.97      0.97      1607

    accuracy   

In [4]:
import copy
import os.path
import sys

pwd = os.getcwd()
sys.path.append(os.path.dirname(pwd))

from utils.model_utils.evaluate import evaluate_model
from utils.model_utils.load_model import *
from utils.model_utils.model_config import ModelConfig
from utils.dataset_utils.load_dataset import load_data
from utils.decompose_utils.weight_remover import WeightRemoverBert
from utils.decompose_utils.concern_identification import ConcernIdentificationBert
from utils.decompose_utils.tangling_identification import TanglingIdentification
from transformers import AutoConfig
from utils.model_utils.save_module import save_module
from datetime import datetime
from utils.decompose_utils.concern_modularization import ConcernModularizationBert
from utils.decompose_utils.sampling import sampling_class
from utils.dataset_utils.load_dataset import convert_dataset_labels_to_binary, extract_and_convert_dataloader
import torch


model_name = "sadickam/sdg-classification-bert"
model_type = "pretrained"
data = "OSDG"
num_labels = 16


# model_name = "textattack/bert-base-uncased-imdb"
# model_type = "pretrained"
# data = "IMDb"
# num_labels = 2

# model_name = "fabriceyhc/bert-base-uncased-yahoo_answers_topics"
# model_type = "pretrained"
# data = "Yahoo"
# num_labels = 10

for i in range(8, 9):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
    
    checkpoint_name = None
    config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
    model_config = ModelConfig(
        _model_name=model_name,
        _model_type=model_type,
        _data=data,
        _transformer_config=config,
        _checkpoint_name=checkpoint_name,
        _device=device,
    )
    
    
    model, tokenizer, checkpoint = load_classification_model(model_config, train_mode=False)
    
    train_dataloader, valid_dataloader, test_dataloader = load_data(
        model_config, batch_size=32
    )
    
    print("Start Time:" + datetime.now().strftime("%H:%M:%S"))
    print("#Module " + str(i) + " in progress....")
    num_samples = 64
    
    positive_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, True, 4, device=device
    )
    negative_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, False, 4, device=device
    )
    
    all_samples = sampling_class(
        train_dataloader, 200, 20, num_labels, False, 4, device=device
    )
    
    print("origin")
    # evaluate_model(model, model_config, test_dataloader)
    
    module = copy.deepcopy(model)
    wr = WeightRemoverBert(model, p=0.9)
    ci = ConcernIdentificationBert(model, p=0.4)
    ti = TanglingIdentification(model, p=0.5)
    
    print("Start Positive CI sparse")
    
    import matplotlib.pyplot as plt
    import numpy as np
    
    def double_plot(tensor1, tensor2):
        tensor_data1 = tensor1.cpu().detach().numpy()
        flatten_data1 = tensor_data1.flatten()
    
        tensor_data2 = tensor2.cpu().detach().numpy()
        flatten_data2 = tensor_data2.flatten()
        
        fig, ax1 = plt.subplots(figsize=(6, 5))
    
        # Plotting the first histogram
        ax1.hist(flatten_data1, bins=40, edgecolor='black', alpha=0.5, label='concern identification', color='b')
        ax1.set_xlabel('Value')
        ax1.set_ylabel('Frequency', color='b')
        ax1.tick_params(axis='y', labelcolor='b')
        
        # Plotting the second histogram
        ax1.hist(flatten_data2, bins=40, edgecolor='black', alpha=0.5, label='tangling identification', color='y')
        ax1.set_ylabel('Frequency', color='g')
        ax1.tick_params(axis='y', labelcolor='g')
    
        plt.title('Distribution of the weight')
        plt.legend()
        plt.grid(True)
        plt.show()
    
    sparsity_list = []
    prec_list = []
    rec_list = []
    f1_list = []
    
    def print_overlapped_weight(model, module):
        top_ff1 = model.bert.encoder.layer[0].intermediate.dense.weight
        bottom_ff1 = model.bert.encoder.layer[-1].intermediate.dense.weight
        top_ff2 = model.bert.encoder.layer[0].output.dense.weight
        bottom_ff2 = model.bert.encoder.layer[-1].output.dense.weight
        pooler = model.bert.pooler.dense.weight
        classifier = model.bert.pooler.dense.weight
    
        top_ff1_m = module.bert.encoder.layer[0].intermediate.dense.weight
        bottom_ff1_m = module.bert.encoder.layer[-1].intermediate.dense.weight
        top_ff2_m = module.bert.encoder.layer[0].output.dense.weight
        bottom_ff2_m = module.bert.encoder.layer[-1].output.dense.weight
        pooler_m = module.bert.pooler.dense.weight
        classifier_m = module.bert.pooler.dense.weight
    
        double_plot(top_ff1, top_ff1_m)
        double_plot(bottom_ff1, bottom_ff1_m)
        double_plot(top_ff2, top_ff2_m)
        double_plot(bottom_ff2, bottom_ff2_m)
        double_plot(pooler, pooler_m)
        double_plot(classifier, classifier_m)
    
    def get_sparsity(module):
        cnt = 0
        for i, layer in enumerate(module.bert.encoder.layer):
            cnt += torch.sum(layer.intermediate.dense.weight == 0).item()
            cnt += torch.sum(layer.output.dense.weight == 0).item()
        cnt += torch.sum(module.bert.pooler.dense.weight == 0).item()
        cnt += torch.sum(module.classifier.weight == 0).item()
        return cnt
    
    def get_report(result, i):
        import re
        if not isinstance(result, str):
            result = str(result)
        pattern = rf'\s+{i}\s+(\d\.\d+)\s+(\d\.\d+)\s+(\d\.\d+)\s+(\d+)'
    
        match = re.search(pattern, result)
        precision = float(match.group(1))
        recall = float(match.group(2))
        f1_score = float(match.group(3))
        return precision, recall, f1_score
    
    eval_step = 1
    for idx, batch in enumerate(all_samples):
        input_ids, attn_mask, _, total_sampled = batch
        ref_module = copy.deepcopy(module)
        with torch.no_grad():
            wr.propagate(module, input_ids)
        if idx % eval_step == 0:
            result = evaluate_model(module, model_config, test_dataloader)
            sparsity_list.append(get_sparsity(module))
            prec, rec, f1 = get_report(result, i)
            prec_list.append(prec)
            rec_list.append(prec)
            f1_list.append(prec)
    print("Start Positive CI after sparse")
    
    for idx, batch in enumerate(positive_samples):
        ref_module = copy.deepcopy(module)
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            ci.propagate(module, input_ids)
        if idx % eval_step == 0:
            result = evaluate_model(module, model_config, test_dataloader)
            sparsity_list.append(get_sparsity(module))
            prec, rec, f1 = get_report(result, i)
            prec_list.append(prec)
            rec_list.append(prec)
            f1_list.append(prec)
    
    print("Start Negative TI")
    
    for idx, batch in enumerate(negative_samples):
        ref_module = copy.deepcopy(module)
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            ti.propagate(module, input_ids)
        if idx % eval_step == 0:
            result = evaluate_model(module, model_config, test_dataloader)
            sparsity_list.append(get_sparsity(module))
            prec, rec, f1 = get_report(result, i)
            prec_list.append(prec)
            rec_list.append(prec)
            f1_list.append(prec)
    
    print(sparsity_list)
    print(prec_list)
    print(rec_list)
    print(f1_list)
    
    # import matplotlib.pyplot as plt
    
    # def plot_sparsity(sparsity_list):
    #     """
    #     모델의 sparsity 그래프를 그리는 함수.
        
    #     Args:
    #     sparsity_list (list of float): sparsity 값들의 리스트.
    #     """
    #     plt.figure(figsize=(10, 6))
    #     plt.plot(sparsity_list, marker='o', linestyle='-', color='b')
    #     plt.title('Model Sparsity Over Time')
    #     plt.xlabel('Iteration')
    #     plt.ylabel('Sparsity')
    #     plt.grid(True)
    #     plt.show()
    
    # plot_sparsity(sparsity_list)





Directory /home/Minwoo/LESN/Decompose/DecomposeBERT/Models/Configs/pretrained/sadickam/sdg-classification-bert exists.
Loading the model.
Start Time:17:29:20
#Module 8 in progress....
origin
Start Positive CI sparse


Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [02:30<00:00,  2.66it/s]


Loss: 0.9465
Precision: 0.7790, Recall: 0.7857, F1-Score: 0.7781
              precision    recall  f1-score   support

           0       0.76      0.67      0.71       797
           1       0.84      0.71      0.77       775
           2       0.87      0.88      0.88       795
           3       0.87      0.82      0.85      1110
           4       0.86      0.80      0.83      1260
           5       0.88      0.69      0.77       882
           6       0.85      0.80      0.82       940
           7       0.48      0.61      0.54       473
           8       0.65      0.86      0.74       746
           9       0.61      0.72      0.66       689
          10       0.76      0.78      0.77       670
          11       0.62      0.81      0.71       312
          12       0.72      0.81      0.76       665
          13       0.83      0.86      0.84       314
          14       0.86      0.78      0.82       756
          15       0.97      0.98      0.97      1607

    accuracy   

Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [02:43<00:00,  2.45it/s]


Loss: 0.9466
Precision: 0.7789, Recall: 0.7857, F1-Score: 0.7781
              precision    recall  f1-score   support

           0       0.76      0.67      0.71       797
           1       0.84      0.71      0.77       775
           2       0.87      0.88      0.88       795
           3       0.87      0.82      0.85      1110
           4       0.86      0.80      0.83      1260
           5       0.88      0.69      0.77       882
           6       0.86      0.80      0.82       940
           7       0.48      0.61      0.54       473
           8       0.65      0.86      0.74       746
           9       0.61      0.72      0.66       689
          10       0.76      0.78      0.77       670
          11       0.62      0.81      0.71       312
          12       0.72      0.81      0.76       665
          13       0.83      0.86      0.84       314
          14       0.86      0.78      0.82       756
          15       0.97      0.98      0.97      1607

    accuracy   

...

Loss: 0.9156
Precision: 0.7713, Recall: 0.7714, F1-Score: 0.7664
              precision    recall  f1-score   support

           0       0.72      0.65      0.69       797
           1       0.86      0.66      0.75       775
           2       0.85      0.87      0.86       795
           3       0.88      0.80      0.84      1110
           4       0.85      0.81      0.83      1260
           5       0.90      0.68      0.78       882
           6       0.84      0.78      0.81       940
           7       0.50      0.55      0.52       473
           8       0.63      0.86      0.73       746
           9       0.53      0.75      0.62       689
          10       0.79      0.74      0.77       670
          11       0.66      0.77      0.71       312
          12       0.69      0.80      0.74       665
          13       0.81      0.86      0.84       314
          14       0.85      0.77      0.81       756
          15       0.97      0.98      0.97      1607

    accuracy   

Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [02:43<00:00,  2.45it/s]


Loss: 0.9156
Precision: 0.7713, Recall: 0.7714, F1-Score: 0.7664
              precision    recall  f1-score   support

           0       0.72      0.65      0.69       797
           1       0.86      0.66      0.75       775
           2       0.85      0.87      0.86       795
           3       0.88      0.80      0.84      1110
           4       0.85      0.81      0.83      1260
           5       0.90      0.68      0.78       882
           6       0.84      0.78      0.81       940
           7       0.50      0.55      0.52       473
           8       0.63      0.86      0.73       746
           9       0.53      0.75      0.62       689
          10       0.79      0.74      0.77       670
          11       0.66      0.77      0.71       312
          12       0.69      0.80      0.74       665
          13       0.81      0.86      0.84       314
          14       0.85      0.77      0.81       756
          15       0.97      0.98      0.97      1607

    accuracy   

Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [02:43<00:00,  2.45it/s]

Loss: 0.9156
Precision: 0.7713, Recall: 0.7714, F1-Score: 0.7664
              precision    recall  f1-score   support

           0       0.72      0.65      0.69       797
           1       0.86      0.66      0.75       775
           2       0.85      0.87      0.86       795
           3       0.88      0.80      0.84      1110
           4       0.85      0.81      0.83      1260
           5       0.90      0.68      0.78       882
           6       0.84      0.78      0.81       940
           7       0.50      0.55      0.52       473
           8       0.63      0.86      0.73       746
           9       0.53      0.75      0.62       689
          10       0.79      0.74      0.77       670
          11       0.66      0.77      0.71       312
          12       0.69      0.80      0.74       665
          13       0.81      0.86      0.84       314
          14       0.85      0.77      0.81       756
          15       0.97      0.98      0.97      1607

    accuracy   